In [17]:
import pandas as pd
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import os
import dask.dataframe as ddf

import sys
sys.path.append("..")

from channeling_lib import AWS_file_loader, load_path

stations_str = ['Tom Joad','Rosanna','Bette Davis', 'Layla', 'Mrs Robinson']

TinyTag_str = ['CEB_1']

In [18]:
path = load_path()+'RawData/TinyTag/'

instrument_textbook_name = load_path()+'instrument_textbook_BLcourse_spring2025(BL instruments).csv'
instrument_textbook_data = pd.read_csv(instrument_textbook_name, encoding='latin1')

# Loading start and end times of Tinytag for calibration

In [19]:
instrument_textbook_data

,Station name,Category,Setup by group,Quantities measured,LAT (dec deg),LON (dec deg),elevation from GPS (masl),elevation from DEM (masl),Radiometer height(m),Th height (m),Anemometer height (m),CEB orientation (Tg),Setup time (UTC),Retrieval time (UTC),Maintenance start time (UTC),Maintenance duration (minutes),Comments,Unnamed: 17
0,Layla,AWS-P,3,"T, Ws, Wd, Tg, P",78.172686,16.010102,186 (NOT ACCURATE),28.2,NaN,1.820,2.09,NE,1/26/25 11:25,2/3/25 11:10,1/30/25 9:40,30.0,NaN,NaN
1,TH3,Tinytag,3,"T, Rh",78.181669,15.908088,181 (NOT ACCURATE),20.9,NaN,1.400,NaN,NaN,1/26/25 12:30,2/3/25 11:54,1/30/25 10:15,10.0,NaN,NaN
2,TH2,Tinytag,3,"T, Rh",78.212169,15.756418,160 (NOT ACCURATE),1.0,NaN,1.650,NaN,NaN,1/26/25 12:45,2/3/25 12:04,1/30/25 10:35,15.0,NaN,NaN
3,Tom Joad,AWS,3,"T, Ws, Wd, Tg",78.226178,15.650295,NaN,0.5,NaN,1.900,2.21,N,1/26/25 14:30,2/3/25 12:14,1/30/25 10:50,15.0,NaN,NaN
4,Rosanna,AWS-P,3,"T, Ws, Wd, Tg, P",78.239350,15.749290,NaN,1.4,NaN,1.680,2.05,NE,1/27/25 10:30,2/3/25 9:10,1/31/25 10:50,40.0,NaN,NaN
5,TH5,Tinytag,3,"T, Rh",78.223609,15.797014,NaN,3.6,NaN,1.650,NaN,NaN,1/27/25 11:10,2/3/25 9:44,1/31/25 12:17,6.0,NaN,NaN
6,Bette Davis,AWS,3,"T, Ws, Wd, Tg",78.213555,15.857184,53 (NOT ACCURATE),4.4,NaN,1.700,2.08,N,1/27/25 12:10,2/3/25 9:52,1/31/25 13:05,53.0,NaN,NaN
7,TH6,Tinytag,3,"T, Rh",78.197500,15.959215,72 (NOT ACCURATE),10.9,NaN,1.570,NaN,NaN,1/27/25 13:30,2/3/25 10:23,1/31/25 15:05,8.0,NaN,NaN
8,Mrs Robinson,AWS,3,"T, Ws, Wd, Tg",78.192157,16.029810,76 (NOT ACCURATE),17.3,NaN,1.780,2.09,W,1/27/25 13:40,2/3/25 10:37,1/31/25 16:00,30.0,NaN,NaN
9,TH1,TinyTag,1,"T, Rh",78.240911,15.354190,265 (NOT ACCURATE),27.5,NaN,1.490,NaN,NaN,1/27/25 10:58,2/3/25 12:48,NaN,NaN,NaN,tiny tag between fjord station and Björndalen


In [20]:
manual_times = {}


for station in stations_str:
    setup_time_idx = pd.to_datetime(instrument_textbook_data.loc[instrument_textbook_data['Station name'] == station, 'Setup time (UTC)'].values[0])

    maintenance_start_time_idx = pd.to_datetime(instrument_textbook_data.loc[instrument_textbook_data['Station name'] == station, 'Maintenance start time (UTC)'].values[0])

    maintenance_duration_idx = int(instrument_textbook_data.loc[instrument_textbook_data['Station name'] == station, 'Maintenance duration (minutes)'].values[0])

    retrieval_time_idx = pd.to_datetime(instrument_textbook_data.loc[instrument_textbook_data['Station name'] == station, 'Retrieval time (UTC)'].values[0])

    manual_times[station] = {
            'setup_time': setup_time_idx,
            'maintenance_start_time': maintenance_start_time_idx,
            'maintenance_duration': maintenance_duration_idx,
            'retrieval_time': retrieval_time_idx
        }

# Loading TinyTag Calibration data

The calibration data is stored in format:

- CEB_i_calibration_data

- THi_calibration_data

- TTi_calibration_data

Where i corresponds to the number of the TinyTag, i.e. the same number as the raw files

In [21]:
#function from unis github
def read_Tinytag(filename, sensor):
    '''
    Reads data from one or several data files from the Tinytag output files.

    Parameters:
    -------
    filename: str
        String with path to file(s)
        If several files shall be read, specify a string including UNIX-style wildcards
    sensor: str
        One of "TT", "TH" or "CEB"
    Returns
    -------
    df : pandas dataframe
        a pandas dataframe with time as index and the individual variables as columns.
    '''

    import dask.dataframe as ddf


    if sensor == "TT":
        df = ddf.read_csv(filename, delimiter="\t", skiprows=5, parse_dates=[1], date_format="%d %b %Y %H:%M:%S", names=["RECORD", "TIMESTAMP", "T_black", "T_white"], encoding = "ISO-8859-1")
    elif sensor == "TH":
        df = ddf.read_csv(filename, delimiter="\t", skiprows=5, parse_dates=[1], date_format="%d %b %Y %H:%M:%S", names=["RECORD", "TIMESTAMP", "T", "RH"], encoding = "ISO-8859-1")
    elif sensor == "CEB":
        df = ddf.read_csv(filename, delimiter="\t", skiprows=5, parse_dates=[1], date_format="%d %b %Y %H:%M:%S", names=["RECORD", "TIMESTAMP", "T"], encoding = "ISO-8859-1")
    else:
        assert False, 'Sensortype of Tinytag not known. Should be one of "TT", "TH" or "CEB".'

    df = df.compute()
    df.set_index("TIMESTAMP", inplace=True)

    for key in list(df.columns):
        if key == "RECORD":
            pass
        else:
            data = [float(i.split(" ")[0]) for i in df[key]]
            unit = df[key].iloc[0].split(" ")[1]
            if unit == "°C":
                unit = "degC"
            new_key = f"{key}_{unit}"

            df[new_key] = data

            df.drop(key, axis=1, inplace=True)

    return df

In [22]:
# Get all CEB_i folders inside TinyTag
folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith('CEB_')]

# Dictionary to store the datasets
calibration_data = {}

# Loop through each CEB_i folder
for folder in folders:
    folder_path = os.path.join(path, folder)
    
    # Get all .txt files that start with "CEB_i_calibration_"
    files = [f for f in os.listdir(folder_path) if f.startswith(folder + "_calibration_") and f.endswith(".txt")]
    
    for file in files:
        file_path = os.path.join(folder_path, file)
        
        # Run the read_Tinytag function
        dataset = read_Tinytag(file_path, 'CEB')
        
        # Store the dataset in a dictionary using the folder name as the key
        calibration_data[f"{folder}_calibration_data"] = dataset

# Loop over the stored datasets and assign them as individual variables
for dataset_name in calibration_data.keys():
    globals()[dataset_name] = calibration_data[dataset_name]

In [23]:
# TH
folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith('TH')]

# Dictionary to store the datasets
calibration_data = {}

for folder in folders:
    folder_path = os.path.join(path, folder)
    
    # Get all .txt files that start with "THi_calibration_"
    files = [f for f in os.listdir(folder_path) if f.startswith(folder + "_calibration_") and f.endswith(".txt")]
    
    for file in files:
        file_path = os.path.join(folder_path, file)
        
        # Run the read_Tinytag function
        dataset = read_Tinytag(file_path, 'TH')
        
        # Store the dataset in a dictionary using the folder name as the key
        calibration_data[f"{folder}_calibration_data"] = dataset

for dataset_name in calibration_data.keys():
    globals()[dataset_name] = calibration_data[dataset_name]

In [24]:
# TT
folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith('TT')]

# Dictionary to store the datasets
calibration_data = {}

for folder in folders:
    folder_path = os.path.join(path, folder)
    
    # Get all .txt files that start with "TTi_calibration_"
    files = [f for f in os.listdir(folder_path) if f.startswith(folder + "_calibration_") and f.endswith(".txt")]
    
    for file in files:
        file_path = os.path.join(folder_path, file)
        
        # Run the read_Tinytag function
        dataset = read_Tinytag(file_path, 'TT')
        
        # Store the dataset in a dictionary using the folder name as the key
        calibration_data[f"{folder}_calibration_data"] = dataset

for dataset_name in calibration_data.keys():
    globals()[dataset_name] = calibration_data[dataset_name]

# Loading TinyTag data

In [25]:
# Get all CEB_i folders inside TinyTag
folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f)) and f.startswith('CEB_')]

# Dictionary to store the datasets
calibration_data = {}

# Loop through each CEB_i folder
for folder in folders:
    folder_path = os.path.join(path, folder)
    
    # Get all .txt files that does not include "CEB_i_calibration_"
    # files = [f for f in os.listdir(folder_path) if f.startswith(folder + "_calibration_") and f.endswith(".txt")]
    files = [f for f in os.listdir(folder_path) if f.endswith(".txt") and "_calibration_" not in f]

    
    for file in files:
        file_path = os.path.join(folder_path, file)
        
        # Run the read_Tinytag function
        dataset = read_Tinytag(file_path, 'CEB')
        
        # Store the dataset in a dictionary using the folder name as the key
        calibration_data[f"{folder}_calibration_data"] = dataset

# Loop over the stored datasets and assign them as individual variables
for dataset_name in calibration_data.keys():
    globals()[dataset_name] = calibration_data[dataset_name]